In [ ]:
# !pip install cfgrib s3fs h5netcdf h5pyd

In [50]:
# Setup
import os
import pandas as pd
import numpy as np
import xarray as xr
import datetime
import io
import requests
import s3fs

start_time = "2022-06-01 00:00"
times = 24*30 # 1 month of data (20 train, 10 forecast)

In [3]:
# Read in list of RAWS Stations
df = pd.read_csv("raws_stations_WA.csv")

# Filter to those with complete fmda data
df = df[(df[['air_temp', 'relative_humidity', 'precip_accum',
       'fuel_moisture', 'wind_speed', 'solar_radiation']]==1).sum(axis=1)==6]

# Get list of coords
points = list(df[["lon","lat"]].itertuples(index=False,name=None))
names = np.unique(df['STID'])

print(df.shape)

(40, 9)


In [4]:
# Create a range of dates, June 1 to June 30
dates = pd.date_range(
    start=start_time,
    periods=times, # 1 month of data
    freq="1H",
)

In [60]:
base_url = "https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20140730/conus/hrrr.t18z.wrfsubhf0015.grib2"
print(base_url)

In [59]:
date = "20140730"
model = "t18z.wrfsubhf0015"
url = "https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr."+date+"/conus/hrrr."+model+".grib2"

print(url)

https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20140730/conus/hrrr.t18z.wrfsubhf0015.grib2


In [58]:
!wget https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20140730/conus/hrrr.t18z.wrfsubhf0015.grib2 -O data/hrrr.20140730.grib2

--2023-05-24 16:52:27--  https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr.20140730/conus/hrrr.t18z.wrfsubhf0015.grib2
Resolving noaa-hrrr-bdp-pds.s3.amazonaws.com... 3.5.0.129, 3.5.25.180, 52.216.82.200, ...
Connecting to noaa-hrrr-bdp-pds.s3.amazonaws.com|3.5.0.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22567105 (22M) [application/octet-stream]
Saving to: 'data/hrrr.20140730.grib2'

     0K .......... .......... .......... .......... ..........  0%  320K 69s
    50K .......... .......... .......... .......... ..........  0% 64.1M 34s
   100K .......... .......... .......... .......... ..........  0% 1008K 30s
   150K .......... .......... .......... .......... ..........  0% 34.6M 23s
   200K .......... .......... .......... .......... ..........  1% 50.9M 18s
   250K .......... .......... .......... .......... ..........  1%  931K 19s
   300K .......... .......... .......... .......... ..........  1% 21.3M 16s
   350K .......... .......... ..........

In [ ]:
ds = xr.load_dataset("/hrrr.t18z.wrfsubhf0015.grib2")